# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [1]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [2]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [3]:
def precision_at_k(true_list, predicted_list, k):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    if len(predicted_list) == 0:
        return 0.0
    return round(len([1 for doc_id in predicted_list if doc_id in true_set]) / len(predicted_list), 3)
def recall_at_k(true_list, predicted_list, k):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    if len(true_set) < 1:
        return 1.0
    return round(len([1 for doc_id in predicted_list if doc_id in true_set]) / len(true_set), 3)
def f1_at_k(true_list, predicted_list, k):
    p = precision_at_k(true_list, predicted_list, k)
    r = recall_at_k(true_list, predicted_list, k)
    if p == 0.0 or r == 0.0:
        return 0.0
    return round(2.0 / (1.0/p + 1.0/r), 3)
def results_quality(true_list, predicted_list):
    p5 = precision_at_k(true_list, predicted_list, 5)
    f1_30 = f1_at_k(true_list, predicted_list, 30)
    if p5 == 0.0 or f1_30 == 0.0:
        return 0.0
    return round(2.0 / (1.0/p5 + 1.0/f1_30), 3)

In [4]:
assert precision_at_k(range(10), [1,2,3] , 2) == 1.0
assert recall_at_k(   range(10), [10,5,3], 2) == 0.1
assert precision_at_k(range(10), []      , 2) == 0.0
assert precision_at_k([],        [1,2,3],  5) == 0.0
assert recall_at_k(   [],        [10,5,3], 2) == 1.0
assert recall_at_k(   range(10), [],       2) == 0.0
assert f1_at_k(       [],        [1,2,3],  5) == 0.0
assert f1_at_k(       range(10), [],       2) == 0.0
assert f1_at_k(       range(10), [0,1,2],  2) == 0.333
assert f1_at_k(       range(50), range(5), 30) == 0.182
assert f1_at_k(       range(50), range(10), 30) == 0.333
assert f1_at_k(       range(50), range(30), 30) == 0.75
assert results_quality(range(50), range(5))  == 0.308
assert results_quality(range(50), range(10)) == 0.5
assert results_quality(range(50), range(30)) == 0.857
assert results_quality(range(50), [-1]*5 + list(range(5,30))) == 0.0

In [6]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below.
url = 'http://9b62-34-123-201-180.ngrok-free.app'

qs_res = []
for q, true_wids in queries.items():
    true_wids = map(int, true_wids)
    duration, ap = None, None
    t_start = time()
    try:
        res = requests.get(url + '/search', {'query': q}, timeout=35)
        # print(q)
        #print('anything')
        duration = time() - t_start
        #print(res.content)
        if res.status_code == 200:
            #print('anything')
            pred_wids, _ = zip(*res.json())
            #print('later')
            ap = average_precision(true_wids, pred_wids,)
    except:
         pass

    qs_res.append((q, duration, ap))
    print((q, duration, ap))


('genetics', 0.4357006549835205, 0.382)
('Who is considered the "Father of the United States"?', 6.036288499832153, 0.0)
('economic', 0.8137991428375244, 0.144)
('When was the United Nations founded?', 3.049494743347168, 0.124)
('video gaming', 1.3163621425628662, 0.333)
('3D printing technology', 0.8334200382232666, 0.036)
('Who is the author of "1984"?', 1.5932774543762207, 0.0)
('bioinformatics', 0.4131286144256592, 0.146)
('Who is known for proposing the heliocentric model of the solar system?', 4.296607971191406, 0.452)
('Describe the process of water erosion.', 2.270473003387451, 0.761)
('When was the Berlin Wall constructed?', 1.3161044120788574, 0.573)
('What is the meaning of the term "Habeas Corpus"?', 1.6618664264678955, 0.555)
('telecommunications', 0.38660597801208496, 0.143)
('internet', 0.71274733543396, 0.109)
('What are the characteristics of a chemical element?', 2.277853488922119, 0.304)
('Describe the structure of a plant cell.', 2.3162946701049805, 0.254)
('Who pai

In [ ]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below.
url = 'http://9b62-34-123-201-180.ngrok-free.app'
# rq=0
qs_res = []
for q, true_wids in queries.items():
  true_wids = map(int, true_wids)
  duration, rq = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q}, timeout=35)
    # print(res.content)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      #print(pred_wids)
      #rq=48324
      rq = results_quality(true_wids, pred_wids)
      qs_res.append((q, duration, rq))
  except:
    pass

  # qs_res.append((q, duration, rq))
  # print((q, duration, rq))